<a href="https://colab.research.google.com/github/soohyunme/TensorFlow_Tutorial/blob/main/Code/03_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [316]:
import os
from threading import active_count

from tensorflow.python.keras.layers.core import Activation
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.datasets import cifar10


# Device setting

In [317]:
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0],True)

# Load data

In [318]:
(x_train,y_train), (x_test,y_test) = cifar10.load_data()

# Normalize

In [319]:
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Initialize model

In [320]:
model = keras.Sequential(
    [
     keras.Input(shape=(32,32,3)),
     layers.Conv2D(32, 3, padding='valid', activation='relu'),
     layers.MaxPooling2D(pool_size=(2,2)),
     layers.Conv2D(64, 3, activation='relu'),
     layers.MaxPooling2D(pool_size=(2,2)),
     layers.Conv2D(128, 3, activation='relu'),
     layers.Flatten(),
     layers.Dense(64,activation='relu'),
     layers.Dense(10),

    ]
)

In [321]:
print(model.summary())

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_171 (Conv2D)          (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_79 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_172 (Conv2D)          (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_80 (MaxPooling (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_173 (Conv2D)          (None, 4, 4, 128)         73856     
_________________________________________________________________
flatten_54 (Flatten)         (None, 2048)              0         
_________________________________________________________________
dense_106 (Dense)            (None, 64)              

In [322]:
def my_model():
  with tf.device('/device:GPU:0'):
      inputs = keras.Input(shape=(32, 32, 3))
      x = layers.Conv2D(32, 3)(inputs)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      x = layers.MaxPooling2D()(x)
      
      x = layers.Conv2D(64, 3)(x)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      x = layers.MaxPooling2D()(x)
      
      x = layers.Conv2D(128, 3)(x)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      
      x = layers.Flatten()(x)
      x = layers.Dense(64, activation="relu")(x)
      outputs = layers.Dense(10)(x)
      model = keras.Model(inputs=inputs, outputs=outputs)
      return model

## New model


In [323]:
def new_model():
   with tf.device('/device:GPU:0'):
      inputs = keras.Input(shape=(32,32,3))
      x = layers.Conv2D(32,3)(inputs)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      x = layers.MaxPooling2D()(x)
      x = layers.Dropout(0.2)(x)
      
      x = layers.Conv2D(64, 3, padding='same')(x)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      x = layers.Dropout(0.2)(x)

      x = layers.Conv2D(128,3)(x)
      x = layers.BatchNormalization()(x)
      x = keras.activations.relu(x)
      x = layers.Dropout(0.2)(x)
      
      x = layers.Flatten()(x)
      x = layers.Dense(64, activation='relu')(x)
      outputs = layers.Dense(10)(x)
      model = keras.Model(inputs=inputs, outputs=outputs)
      return model


In [324]:
# model = my_model()
model = new_model()

In [325]:
print(model.summary())


Model: "model_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_57 (InputLayer)        [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d_174 (Conv2D)          (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_128 (Bat (None, 30, 30, 32)        128       
_________________________________________________________________
tf.nn.relu_128 (TFOpLambda)  (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_81 (MaxPooling (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_25 (Dropout)         (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_175 (Conv2D)          (None, 15, 15, 64)        184

# Model compile

In [326]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(lr=3e-4),
    metrics=['accuracy'],
)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


# Train and evaluate

In [327]:
with tf.device('/device:GPU:0'):
  history = model.fit(x_train, y_train, batch_size=64, epochs=30, verbose=2)
  model.evaluate(x_test, y_test, batch_size=64, verbose=2)

Epoch 1/30
782/782 - 5s - loss: 1.4617 - accuracy: 0.4830
Epoch 2/30
782/782 - 4s - loss: 1.1058 - accuracy: 0.6095
Epoch 3/30
782/782 - 4s - loss: 0.9579 - accuracy: 0.6614
Epoch 4/30
782/782 - 4s - loss: 0.8601 - accuracy: 0.6982
Epoch 5/30
782/782 - 4s - loss: 0.7936 - accuracy: 0.7218
Epoch 6/30
782/782 - 4s - loss: 0.7387 - accuracy: 0.7406
Epoch 7/30
782/782 - 4s - loss: 0.7034 - accuracy: 0.7543
Epoch 8/30
782/782 - 4s - loss: 0.6573 - accuracy: 0.7697
Epoch 9/30
782/782 - 4s - loss: 0.6240 - accuracy: 0.7826
Epoch 10/30
782/782 - 4s - loss: 0.5936 - accuracy: 0.7915
Epoch 11/30
782/782 - 4s - loss: 0.5659 - accuracy: 0.8001
Epoch 12/30
782/782 - 4s - loss: 0.5353 - accuracy: 0.8103
Epoch 13/30
782/782 - 4s - loss: 0.5115 - accuracy: 0.8215
Epoch 14/30
782/782 - 4s - loss: 0.4958 - accuracy: 0.8233
Epoch 15/30
782/782 - 4s - loss: 0.4677 - accuracy: 0.8349
Epoch 16/30
782/782 - 4s - loss: 0.4561 - accuracy: 0.8384
Epoch 17/30
782/782 - 4s - loss: 0.4306 - accuracy: 0.8459
Epoch 

# SUGGESTIONS


1. **What accuracy can you get on the test set by training longer, increasing the model size, changing kernel sizes, etc?**
> Baseline : 0.7197  
> Add MaxPooling each Conv2D : 0.6812  
> More Conv2D layer : 0.7133  
> Change kernel size : 0.6547  
> Change kernel size +  more epochs : 0.6669  
> Increase batch size + more epochs : 0.7007  
> Increase batch size : 0.6738  
> (Baseline) Apply drop out(0.2) + more epochs(30) : 0.7732

2. **In the last video we train a FC on MNIST: what can you get by using a conv net instead?**
> FC보다 파라미터의 수를 줄일 수 있음  
> 이미지의 형상을 고려하여 학습할 수 있음  
> 보다 적은 양의 학습 데이터로도 더 빠른 시간으로 학습이 가능